In [1]:
include("util_simplex.jl")

# Problème d'optimisation linéaire avec l'algorithme du simplexe

Puisque la journée est placée sous le signe de l'optimisation, consacrons nous (enfin) à un problème du type:

<br/>

\begin{align}
  & argmin_x \quad c\cdot x \\
\text{sous contraintes :} \qquad & Ax \leqslant b
\end{align}

In [2]:
A = load(Matrix{Float64}, "dataSimplex/A.dat");
b = load(Vector{Float64}, "dataSimplex/b.dat");
c = load(Vector{Float64}, "dataSimplex/c.dat");

In [3]:
simplex(A, b, c)

ErrorException: Singular matrix for entering index 2

## Analyse en arithmétique rationnelle

In [4]:
const Rat = Rational{BigInt}

res = simplex(Rat.(A), Rat.(b), Rat.(c))
value.(res.solution)

8-element Array{Float64,1}:
 938.6915102244791       
 124.99999999999625      
 938.6915102244419       
   0.0                   
 938.6915102244336       
   3.0947466811426485e-12
 938.6915102244423       
   0.0                   

## Analyse en arithmétique stochastique

In [5]:
function test(T)
    try
        res = simplex(T.(A), T.(b), T.(c))
        value.(res.solution)
    catch e
        value.(NaN*ones(T, length(c)))
    end
end

test (generic function with 1 method)

In [6]:
hcat(test(Rational{BigInt}), test(Float64), (test(SFloat64) for _ in 1:10)...)

8×12 Array{Float64,2}:
 938.692        NaN  NaN  938.692        NaN  …  938.692        938.692      
 125.0          NaN  NaN  125.0          NaN     125.0          125.0        
 938.692        NaN  NaN  938.692        NaN     938.692        938.692      
   0.0          NaN  NaN    0.0          NaN       0.0            0.0        
 938.692        NaN  NaN  938.692        NaN     938.692        938.692      
   3.09475e-12  NaN  NaN    3.08376e-12  NaN  …    3.11218e-12    3.08376e-12
 938.692        NaN  NaN  938.692        NaN     938.692        938.692      
   0.0          NaN  NaN    0.0          NaN       0.0            0.0        

## Analyse de la version "réparée"

In [7]:
fix_simplex = true
hcat(test(Rational{BigInt}), test(Float64), (test(SFloat64) for _ in 1:10)...)

8×12 Array{Float64,2}:
 938.692        938.692        938.692        …  938.692        938.692      
 125.0          125.0          125.0             125.0          125.0        
 938.692        938.692        938.692           938.692        938.692      
   0.0            0.0            0.0               0.0            0.0        
 938.692        938.692        938.692           938.692        938.692      
   3.09475e-12    3.11218e-12    3.05533e-12  …    3.02691e-12    3.08376e-12
 938.692        938.692        938.692           938.692        938.692      
   0.0            0.0            0.0               0.0            0.0        

In [8]:
@reliable_digits simplex(SFloat64.(A), SFloat64.(b), SFloat64.(c)).solution

8-element Array{Tuple{Float64,Float64},1}:
 (938.6915102244791, 15.805888328751902)  
 (124.99999999999628, 15.881820445247028) 
 (938.691510224442, 15.873237615700614)   
 (0.0, NaN)                               
 (938.6915102244336, 15.94238896478375)   
 (3.070965703955153e-12, 2.08132710022463)
 (938.6915102244423, 15.791873966951764)  
 (0.0, NaN)                               

In [9]:
using BenchmarkTools
@btime simplex(A, b, c)
@btime simplex(SFloat64.(A), SFloat64.(b), SFloat64.(c))
@btime simplex(Rat.(A), Rat.(b), Rat.(c));

  52.500 μs (185 allocations: 76.67 KiB)
  206.155 μs (250 allocations: 64.19 KiB)
  27.754 ms (858913 allocations: 18.25 MiB)
